In [1]:
import os
import os.path as osp

import random

from tqdm import tqdm

import numpy as np

import open3d as o3d
import plyfile

import utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [1]:
import numpy as np
from plyfile import PlyData, PlyElement

def npy2ply(points: np.ndarray, colors: np.ndarray):
    ply_points = np.empty(points.shape, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])
    ply_points['x'] = points[:, 0]
    ply_points['y'] = points[:, 1]
    ply_points['z'] = points[:, 2]

# 示例数据：假设有5个点和2条线段
points = np.array([
    (0, 0, 0),
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (1, 1, 1)
], dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])

lines = np.array([
    (0, 1),
    (2, 3)
], dtype=[('vertex1', 'u4'), ('vertex2', 'u4')])

vertices = PlyElement.describe(points, 'vertex')
edges = PlyElement.describe(lines, 'edge')

elements = [vertices, edges]

ply_data = PlyData(elements, text=True)
ply_data.write('output_with_edges.ply')


In [35]:
def join_struct_arrays(arrays):
    sizes = np.array([a.itemsize for a in arrays if a is not None]) # 每个结构化张量中单个元素数据字节数
    offsets = np.r_[0, sizes.cumsum()] # 合并每个结构化张量后单个元素数据字节数序列
    n = len(arrays[0]) # 要求拼接的多个张量具有相同数量的元素数量，这是水平拼接
    joint = np.empty((n, offsets[-1]), dtype=np.uint8)
    for a, size, offset in zip(arrays, sizes, offsets):
        joint[:,offset:offset+size] = a.view(np.uint8).reshape(n,size)
    dtype = sum((a.dtype.descr for a in arrays if a is not None), [])
    return joint.ravel().view(dtype)


# 定义点云数据
points = np.array([
    (0.0, 0.0, 0.0),  # 点1的坐标
    (1.0, 0.0, 0.0),  # 点2的坐标
    (1.0, 1.0, 0.0),  # 点3的坐标
    (0.0, 1.0, 0.0)   # 点4的坐标
], dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])

colors = np.array([
    (255, 0, 0),    # 点1的颜色 (红色)
    (0, 255, 0),    # 点2的颜色 (绿色)
    (0, 0, 255),    # 点3的颜色 (蓝色)
    (255, 255, 0)   # 点4的颜色 (黄色)
], dtype=[('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

ply_vtx = PlyElement.describe(join_struct_arrays([points, colors]), "vertex")

plydata = PlyData([ply_vtx], text=False)
# 将 PlyData 对象写入 PLY 文件
plydata.write('colored_pointcloud.ply')

In [36]:
points = np.random.rand(100, 3)
colors = (np.random.rand(100, 3) * 255.0).astype(np.int32)

points = np.core.records.fromarrays(points.transpose(), dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])
colors = np.core.records.fromarrays(colors.transpose(), dtype=[('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

ply_vtx = PlyElement.describe(join_struct_arrays([points, colors]), "vertex")

plydata = PlyData([ply_vtx], text=False)
# 将 PlyData 对象写入 PLY 文件
plydata.write('colored_pointcloud.ply')